In [1]:
%load_ext autoreload
%autoreload 2

## Prerequisties



following extensions should be enabled in your postgres database
- postgis
- postgis_raster


enable GDAL by executing these queries in your postgres database
```
SET postgis.gdal_enabled_drivers = 'ENABLE_ALL';
SELECT pg_reload_conf();
```


Now restart Postgresql

In [2]:
from postgis2raster import analysis_circle, analysis_polygon

In [3]:
table_name = "gis_osm_roads_free_1"

In [16]:
%%time
analysis_circle(
    table=table_name,
    output_raster='test_circle.tiff',
    query_x=16.55268272,
    query_y=40.82717010,
    radius=2500,
    cell_size=30,
    #classes = ['secondary'],
    class_column = 'fclass',
    geom_column = 'wkb_geometry',
    positive=1,
    negative=0,
    nodata=254
)

4326
0.00023670856264282136
        WITH q AS (                        WITH a AS (                SELECT                     ST_SetSRID(ST_Point(16.55268272, 40.8271701), 4326) AS geom            ),            b AS (                SELECT                     ST_Buffer(ST_Transform(a.geom, 3857), 2500.0) AS geom                FROM a            )            SELECT                ST_Transform(geom, 4326) as geom            FROM b         AS geom        ),        features AS (                        SELECT                                 t.wkb_geometry AS geom                        FROM                                 public.gis_osm_roads_free_1 t,                                q                        WHERE                                 ST_Intersects(q.geom, t.wkb_geometry)                                AND                                1=1                ),        q_ras AS (            SELECT                ST_Union(                    ST_AsRaster(                        q.geom,  

True

In [13]:
%%time
analysis_polygon(
    table=table_name,
    output_raster='test_rectangle1.tiff',
    query_x=16.55268272,
    query_y=40.82717010,
    height=5000,
    width=5000,
    cell_size=30,
    #classes = ['secondary'],
    class_column = 'fclass',
    geom_column = 'wkb_geometry',
    positive=1,
    negative=0
)

4326
0.00023670856264282136
polygon
        WITH q AS (                        SELECT ST_GeomFromText('POLYGON((16.530224837897016 40.810174377629814, 16.530224837897016 40.84416146763811, 16.575140602102994 40.84416146763811, 16.575140602102994 40.810174377629814, 16.530224837897016 40.810174377629814))', 4326)         AS geom        ),        features AS (                        SELECT                                 t.wkb_geometry AS geom                        FROM                                 public.gis_osm_roads_free_1 t,                                q                        WHERE                                 ST_Intersects(q.geom, t.wkb_geometry)                                AND                                1=1                ),        q_ras AS (            SELECT                ST_Union(                    ST_AsRaster(                        q.geom,                        (         SELECT ST_SetBandNoDataValue(ST_MakeEmptyRaster(167, 167, 16.530224837897016, 40.84416

True